In [1]:
%pip install wildlife-datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 2.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install wildlife-tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 19.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.8/427.8 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 8.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install timm

Note: you may need to restart the kernel to use updated packages.


In [5]:
import timm
import numpy as np
from wildlife_datasets.datasets import MacaqueFaces
from wildlife_tools.data import WildlifeDataset
import torchvision.transforms as T
from wildlife_datasets import datasets, splits
from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

In [6]:
import timm
import numpy as np
from PIL import Image
import torchvision.transforms as T
import cv2  # Import OpenCV for SIFT

from sklearn.metrics import precision_score, recall_score, f1_score

from wildlife_tools.data import WildlifeDataset
from wildlife_datasets.datasets import MacaqueFaces

from wildlife_datasets import datasets, splits
from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier
from wildlife_tools.features import SIFTFeatures


from wildlife_tools.data import WildlifeDataset, SplitMetadata
from wildlife_tools.features import SIFTFeatures
from wildlife_tools.similarity import MatchDescriptors
from wildlife_tools.inference import KnnClassifier



**CowDataset**

In [ ]:
# # Download dataset (if not already downloaded)
# datasets.CowDataset.get_data('../data/CowDataset')

# # Load dataset metadata
# metadata_CZoo = datasets.CowDataset('../data/CowDataset')
# transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
# dataset = WildlifeDataset(metadata_CZoo.df, metadata_CZoo.root, transform=transform)

In [ ]:
# Download dataset (if not already downloaded)
datasets.CowDataset.get_data('../data/CowDataset', force = True)

# Load dataset metadata
metadata_CZoo = datasets.CowDataset('../data/CowDataset')

# Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
transform = T.Compose([
    T.Resize([224, 224]),  # Resize the image
    # T.ToPILImage(),  # Convert tensor to PIL Image
    T.Grayscale(),  # Convert to grayscale
    # T.ToTensor(),  # Convert PIL Image to numpy array
    # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
])

# Create datasets with transformations
dataset = WildlifeDataset(metadata_CZoo.df, metadata_CZoo.root, transform=transform)
dataset_database_CZoo = WildlifeDataset(metadata_CZoo.df.iloc[100:], metadata_CZoo.root, transform=transform)
dataset_query_CZoo = WildlifeDataset(metadata_CZoo.df.iloc[:100], metadata_CZoo.root, transform=transform)

# Initialize SIFT extractor
sift = cv2.SIFT_create()
extractor_CZoo = SIFTFeatures()

In [ ]:
query_CZoo, database_CZoo = extractor_CZoo(dataset_query_CZoo), extractor_CZoo(dataset_database_CZoo)

In [ ]:
print(f'First 5 query features shape: {[i.shape for i in query_CZoo[:5]]}')
print(f'First 5 database features shape: {[i.shape for i in database_CZoo[:5]]}')

In [ ]:
similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.8])
sim = similarity(query_CZoo, database_CZoo)[0.8]

print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)

In [ ]:
classifier_CZoo = KnnClassifier(k=1, database_labels=dataset_database_CZoo.labels_string)
predictions_CZoo = classifier_CZoo(sim)

print("Predictions for 100 test Images:-\n",predictions_CZoo)
accuracy_CZoo = np.mean(dataset_query_CZoo.labels_string == predictions_CZoo)
print("Accuracy on MacaqueFaces data: {:.2f}%".format(accuracy_CZoo * 100))



In [ ]:
# Nearest neigbour classifier using the similarity
classifier = KnnClassifier(k=1, database_labels=dataset_database_CZoo.labels_string)
preds = classifier(sim)
print("Prediction \t", preds)
print("Ground truth \t", dataset_query_CZoo.labels_string)

acc = sum(preds == dataset_query_CZoo.labels_string) / len(dataset_query_CZoo.labels_string)
print('\n Accuracy: ', acc*100,"%")

**CZoo Dataset**

In [ ]:
# Download dataset (if not already downloaded)
datasets.CZoo.get_data('../data/CZoo')

# Load dataset metadata
metadata_CZoo = datasets.CZoo('../data/CZoo')

# Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
transform = T.Compose([
    T.Resize([224, 224]),  # Resize the image
    # T.ToPILImage(),  # Convert tensor to PIL Image
    T.Grayscale(),  # Convert to grayscale
    # T.ToTensor(),  # Convert PIL Image to numpy array
    # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
])

# Create datasets with transformations
dataset = WildlifeDataset(metadata_CZoo.df, metadata_CZoo.root, transform=transform)
dataset_database_CZoo = WildlifeDataset(metadata_CZoo.df.iloc[100:], metadata_CZoo.root, transform=transform)
dataset_query_CZoo = WildlifeDataset(metadata_CZoo.df.iloc[:100], metadata_CZoo.root, transform=transform)

# Initialize SIFT extractor
sift = cv2.SIFT_create()
extractor_CZoo = SIFTFeatures()

query_CZoo, database_CZoo = extractor_CZoo(dataset_query_CZoo), extractor_CZoo(dataset_database_CZoo)

print(f'First 5 query features shape: {[i.shape for i in query_CZoo[:5]]}')
print(f'First 5 database features shape: {[i.shape for i in database_CZoo[:5]]}')


In [ ]:
similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.8])
sim = similarity(query_CZoo, database_CZoo)[0.8]

print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)


classifier_CZoo = KnnClassifier(k=1, database_labels=dataset_database_CZoo.labels_string)
predictions_CZoo = classifier_CZoo(sim)

print("Predictions for 100 test Images:-\n",predictions_CZoo)
accuracy_CZoo = np.mean(dataset_query_CZoo.labels_string == predictions_CZoo)
print("Accuracy on CZoo data: {:.2f}%".format(accuracy_CZoo * 100))


In [ ]:
# Nearest neigbour classifier using the similarity
classifier = KnnClassifier(k=1, database_labels=dataset_database_CZoo.labels_string)
preds = classifier(sim)
print("Prediction \t", preds)
print("Ground truth \t", dataset_query_CZoo.labels_string)

acc = sum(preds == dataset_query_CZoo.labels_string) / len(dataset_query_CZoo.labels_string)
print('\n Accuracy: ', acc*100,"%")


**IPanda50 Dataset**

In [ ]:
# # Download dataset (if not already downloaded)
# datasets.IPanda50.get_data('../data/IPanda50')
# # Load dataset metadata
# metadata = datasets.IPanda50('../data/IPanda50')
# transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
# dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)

In [ ]:
# Download dataset (if not already downloaded)
datasets.IPanda50.get_data('../data/Ipanda50')

# Load dataset metadata
metadata_Ipanda50 = datasets.IPanda50('../data/Ipanda50')

# Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
transform = T.Compose([
    T.Resize([224, 224]),  # Resize the image
    # T.ToPILImage(),  # Convert tensor to PIL Image
    T.Grayscale(),  # Convert to grayscale
    # T.ToTensor(),  # Convert PIL Image to numpy array
    # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
])

# Create datasets with transformations
dataset = WildlifeDataset(metadata_Ipanda50.df, metadata_Ipanda50.root, transform=transform)
dataset_database_Ipanda50 = WildlifeDataset(metadata_Ipanda50.df.iloc[100:], metadata_Ipanda50.root, transform=transform)
dataset_query_Ipanda50 = WildlifeDataset(metadata_Ipanda50.df.iloc[:100], metadata_Ipanda50.root, transform=transform)

# Initialize SIFT extractor
sift = cv2.SIFT_create()
extractor_Ipanda50 = SIFTFeatures()

query_Ipanda50, database_Ipanda50 = extractor_Ipanda50(dataset_query_Ipanda50), extractor_Ipanda50(dataset_database_Ipanda50)

print(f'First 5 query features shape: {[i.shape for i in query_Ipanda50[:5]]}')
print(f'First 5 database features shape: {[i.shape for i in database_Ipanda50[:5]]}')

In [ ]:
similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.8])
sim = similarity(query_Ipanda50, database_Ipanda50)[0.8]

print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)


classifier_Ipanda50 = KnnClassifier(k=1, database_labels=dataset_database_Ipanda50.labels_string)
predictions_Ipanda50 = classifier_Ipanda50(sim)

print("Predictions for 100 test Images:-\n",predictions_Ipanda50)
accuracy_Ipanda50 = np.mean(dataset_query_Ipanda50.labels_string == predictions_Ipanda50)
print("Accuracy on Ipanda50 data: {:.2f}%".format(accuracy_Ipanda50 * 100))

In [ ]:
# Nearest neigbour classifier using the similarity
classifier = KnnClassifier(k=1, database_labels=dataset_database_Ipanda50.labels_string)
preds = classifier(sim)
print("Prediction \t", preds)
print("Ground truth \t", dataset_query_Ipanda50.labels_string)

acc = sum(preds == dataset_query_Ipanda50.labels_string) / len(dataset_query_Ipanda50.labels_string)
print('\n Accuracy: ', acc*100,"%")


**LionData Dataset**

In [ ]:
# Download dataset (if not already downloaded)
datasets.LionData.get_data('../data/LionData')

# Load dataset metadata
metadata_LionData = datasets.LionData('../data/LionData')

# Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
transform = T.Compose([
    T.Resize([224, 224]),  # Resize the image
    # T.ToPILImage(),  # Convert tensor to PIL Image
    T.Grayscale(),  # Convert to grayscale
    # T.ToTensor(),  # Convert PIL Image to numpy array
    # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
])

# Create datasets with transformations
dataset = WildlifeDataset(metadata_LionData.df, metadata_LionData.root, transform=transform)
dataset_database_LionData = WildlifeDataset(metadata_LionData.df.iloc[100:], metadata_LionData.root, transform=transform)
dataset_query_LionData = WildlifeDataset(metadata_LionData.df.iloc[:100], metadata_LionData.root, transform=transform)

# Initialize SIFT extractor
sift = cv2.SIFT_create()
extractor_LionData = SIFTFeatures()

query_LionData, database_LionData = extractor_LionData(dataset_query_LionData), extractor_LionData(dataset_database_LionData)

print(f'First 5 query features shape: {[i.shape for i in query_LionData[:5]]}')
print(f'First 5 database features shape: {[i.shape for i in database_LionData[:5]]}')

In [ ]:
similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.6])
sim = similarity(query_LionData, database_LionData)[0.6]

print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)


classifier_LionData = KnnClassifier(k=1, database_labels=dataset_database_LionData.labels_string)
predictions_LionData = classifier_LionData(sim)

print("Predictions for 100 test Images:-\n",predictions_LionData)
accuracy_LionData = np.mean(dataset_query_LionData.labels_string == predictions_LionData)
print("Accuracy on LionData data: {:.2f}%".format(accuracy_LionData * 100))


In [ ]:
# Nearest neigbour classifier using the similarity
classifier = KnnClassifier(k=1, database_labels=dataset_database_LionData.labels_string)
preds = classifier(sim)
print("Prediction \t", preds)
print("Ground truth \t", dataset_query_LionData.labels_string)

acc = sum(preds == dataset_query_LionData.labels_string) / len(dataset_query_LionData.labels_string)
print('\n Accuracy: ', acc*100,"%")


**MacaqueFaces Dataset**


In [9]:
# Download dataset (if not already downloaded)
datasets.MacaqueFaces.get_data('../data/MacaqueFaces')

# Load dataset metadata
metadata_MacaqueFaces = datasets.MacaqueFaces('../data/MacaqueFaces')

# Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
transform = T.Compose([
    T.Resize([224, 224]),  # Resize the image
    # T.ToPILImage(),  # Convert tensor to PIL Image
    T.Grayscale(),  # Convert to grayscale
    # T.ToTensor(),  # Convert PIL Image to numpy array
    # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
])

# Create datasets with transformations
dataset = WildlifeDataset(metadata_MacaqueFaces.df, metadata_MacaqueFaces.root, transform=transform)

dataset_database_MacaqueFaces = WildlifeDataset(metadata_MacaqueFaces.df.iloc[100:], metadata_MacaqueFaces.root, transform=transform)
dataset_query_MacaqueFaces = WildlifeDataset(metadata_MacaqueFaces.df.iloc[:100], metadata_MacaqueFaces.root, transform=transform)

# Initialize SIFT extractor
sift = cv2.SIFT_create()
extractor_MacaqueFaces = SIFTFeatures()

query_MacaqueFaces, database_MacaqueFaces = extractor_MacaqueFaces(dataset_query_MacaqueFaces), extractor_MacaqueFaces(dataset_database_MacaqueFaces)



DATASET MacaqueFaces: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


100%|███████████████████████████████████████████████████████████| 6180/6180 [01:04<00:00, 95.66it/s]


In [10]:
# query_MacaqueFaces

[array([[ 0., 25., 37., ..., 19., 27., 21.],
        [ 2., 64., 25., ...,  3.,  4., 21.],
        [63., 63.,  1., ...,  0.,  0.,  0.],
        ...,
        [37., 77.,  4., ...,  0.,  0.,  0.],
        [10.,  4.,  2., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  4.,  0.,  0.]], dtype=float32),
 array([[  3.,   0.,   0., ...,  23.,  38.,  36.],
        [  7.,   1.,   0., ...,  10.,   1.,   0.],
        [  0.,   5.,  19., ...,  59.,  14.,  30.],
        ...,
        [ 25., 147.,  18., ...,   0.,   0.,   0.],
        [ 58.,  17.,   2., ...,   0.,   0.,   0.],
        [ 62., 150.,  18., ...,   0.,   0.,   0.]], dtype=float32),
 array([[ 17., 142.,  20., ...,   0.,   0.,   0.],
        [  5.,  11.,   1., ...,   3., 110.,  47.],
        [  2.,  11.,  32., ...,   0.,   0.,   0.],
        ...,
        [  0.,  29.,  53., ...,   0.,   0.,   0.],
        [  9.,   0.,   0., ...,   0.,   0.,   0.],
        [ 17.,   1.,   0., ...,   0.,   0.,   0.]], dtype=float32),
 array([[114.,   1.,   0., .

In [11]:
#  database_MacaqueFaces

[None,
 array([[20.,  0.,  0., ...,  0.,  0., 23.],
        [ 0.,  0.,  3., ..., 16.,  2.,  5.],
        [ 0.,  0.,  3., ..., 62., 68.,  1.],
        ...,
        [ 0.,  0.,  0., ...,  2.,  2.,  3.],
        [ 7., 53., 84., ...,  0.,  0.,  0.],
        [10., 42., 11., ...,  1.,  2., 39.]], dtype=float32),
 array([[ 54.,   1.,   0., ...,   0.,   0.,   2.],
        [ 98.,   4.,   2., ...,   1.,   0.,   0.],
        [  6.,   9.,  33., ...,   1.,  11.,  94.],
        ...,
        [  0.,   0.,   0., ...,   1., 127., 127.],
        [  0., 114., 136., ...,   0.,   0.,   0.],
        [159.,   4.,   0., ...,   0.,   0.,   0.]], dtype=float32),
 array([[23.,  7.,  5., ...,  0.,  0.,  0.],
        [ 0.,  0., 25., ...,  0.,  0.,  0.],
        [ 5., 59., 33., ...,  0.,  0.,  0.],
        ...,
        [ 5., 18., 16., ...,  7., 19.,  2.],
        [ 0.,  0., 31., ..., 29., 73., 16.],
        [19., 54., 36., ...,  2., 18., 12.]], dtype=float32),
 array([[ 31.,   0.,   0., ...,   0.,   0.,   0.],
      

In [12]:
print(f'First 5 query features shape: {[i.shape for i in query_MacaqueFaces[:5]]}')
# print(f'First 5 database features shape: {[i.shape for i in database_MacaqueFaces[:50]]}')

First 5 query features shape: [(24, 128), (37, 128), (23, 128), (38, 128), (20, 128)]


AttributeError: 'NoneType' object has no attribute 'shape'

In [13]:
similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.7])
sim = similarity(query_MacaqueFaces, database_MacaqueFaces)[0.7]

print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)


classifier_MacaqueFaces = KnnClassifier(k=1, database_labels=dataset_database_MacaqueFaces.labels_string)
predictions_MacaqueFaces = classifier_MacaqueFaces(sim)

print("Predictions for 100 test Images:-\n",predictions_MacaqueFaces)
accuracy_MacaqueFaces = np.mean(dataset_query_MacaqueFaces.labels_string == predictions_MacaqueFaces)
print("Accuracy on MacaqueFaces data: {:.2f}%".format(accuracy_MacaqueFaces * 100))

100%|████████████████████████████████████████████████████| 618000/618000 [00:47<00:00, 13116.91it/s]

Number of SIFT correspondences after 0.8 ratio test threshold: 
 [[ 0.  2.  6. ...  0.  3.  1.]
 [ 0.  1.  2. ...  1.  1.  0.]
 [ 0.  4.  4. ...  0.  2.  0.]
 ...
 [ 0.  9.  4. ...  0.  2.  1.]
 [ 0.  4. 11. ...  1.  0.  0.]
 [ 0.  6.  9. ...  1.  1.  0.]]
Predictions for 100 test Images:-
 ['Thyme' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Tamara' 'Saphy' 'Dan'
 'Dan' 'Dan' 'Saphy' 'Dan' 'Teal' 'Saphy' 'Dan' 'Dan' 'Dan' 'Sonja' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Serena' 'Dan' 'Dan' 'Saphy' 'Serena'
 'Rupee' 'Dan' 'Dan' 'Rupee' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Tamara' 'Tea' 'Judd' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Rupee' 'Lydia' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Saphy' 'Dan' 'Thyme' 'Dan' 'Meg' 'Rupee'
 'Rupee' 'Judd' 'Dan' 'Rupee' 'Dan' 'Dan' 'Serena' 'Dan' 'Dan' 'Dan'
 'Judd' 'Dan' 'Serena' 'Judd' 'Serena' 'Serena' 'Dan' 'Tamara' 'Dan'
 'Saphy' 'Serena' 'Judd' 'Dan']
Accuracy on MacaqueFaces data


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [14]:
# Nearest neigbour classifier using the similarity
classifier = KnnClassifier(k=1, database_labels=dataset_database_MacaqueFaces.labels_string)
preds = classifier(sim)
print("Prediction \t", preds)
print("Ground truth \t", dataset_query_MacaqueFaces.labels_string)

acc = sum(preds == dataset_query_MacaqueFaces.labels_string) / len(dataset_query_MacaqueFaces.labels_string)
print('\n Accuracy: ', acc*100,"%")

Prediction 	 ['Thyme' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Tamara' 'Saphy' 'Dan'
 'Dan' 'Dan' 'Saphy' 'Dan' 'Teal' 'Saphy' 'Dan' 'Dan' 'Dan' 'Sonja' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Serena' 'Dan' 'Dan' 'Saphy' 'Serena'
 'Rupee' 'Dan' 'Dan' 'Rupee' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Tamara' 'Tea' 'Judd' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Rupee' 'Lydia' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Saphy' 'Dan' 'Thyme' 'Dan' 'Meg' 'Rupee'
 'Rupee' 'Judd' 'Dan' 'Rupee' 'Dan' 'Dan' 'Serena' 'Dan' 'Dan' 'Dan'
 'Judd' 'Dan' 'Serena' 'Judd' 'Serena' 'Serena' 'Dan' 'Tamara' 'Dan'
 'Saphy' 'Serena' 'Judd' 'Dan']
Ground truth 	 ['Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 

/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


**NyalaDataSet Dataset**

In [15]:
# Download dataset (if not already downloaded)
datasets.NyalaData.get_data('../data/NyalaData')

# Load dataset metadata
metadata_NyalaData = datasets.NyalaData('../data/NyalaData')

# Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
transform = T.Compose([
    T.Resize([224, 224]),  # Resize the image
    # T.ToPILImage(),  # Convert tensor to PIL Image
    T.Grayscale(),  # Convert to grayscale
    # T.ToTensor(),  # Convert PIL Image to numpy array
    # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
])

# Create datasets with transformations
dataset = WildlifeDataset(metadata_NyalaData.df, metadata_NyalaData.root, transform=transform)
dataset_database_NyalaData = WildlifeDataset(metadata_NyalaData.df.iloc[100:], metadata_NyalaData.root, transform=transform)
dataset_query_NyalaData = WildlifeDataset(metadata_NyalaData.df.iloc[:100], metadata_NyalaData.root, transform=transform)

# Initialize SIFT extractor
sift = cv2.SIFT_create()
extractor_NyalaData = SIFTFeatures()

query_NyalaData, database_NyalaData = extractor_NyalaData(dataset_query_NyalaData), extractor_NyalaData(dataset_database_NyalaData)

print(f'First 5 query features shape: {[i.shape for i in query_NyalaData[:5]]}')
print(f'First 5 database features shape: {[i.shape for i in database_NyalaData[:5]]}')


DATASET NyalaData: DOWNLOADING STARTED.


main.zip: 495MB [00:20, 24.7MB/s] 


DATASET NyalaData: EXTRACTING STARTED.
DATASET NyalaData: FINISHED.



100%|███████████████████████████████████████████████████████████| 1842/1842 [01:24<00:00, 21.91it/s]

First 5 query features shape: [(738, 128), (857, 128), (720, 128), (756, 128), (621, 128)]
First 5 database features shape: [(727, 128), (829, 128), (679, 128), (613, 128), (651, 128)]


In [16]:
similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.7])
sim = similarity(query_NyalaData, database_NyalaData)[0.7]

print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)


classifier_NyalaData = KnnClassifier(k=1, database_labels=dataset_database_NyalaData.labels_string)
predictions_NyalaData = classifier_NyalaData(sim)

print("Predictions for 100 test Images:-\n",predictions_NyalaData)
accuracy_NyalaData = np.mean(dataset_query_NyalaData.labels_string == predictions_NyalaData)
print("Accuracy on NyalaData data: {:.2f}%".format(accuracy_NyalaData * 100))

100%|██████████████████████████████████████████████████████| 184200/184200 [03:59<00:00, 770.33it/s]

Number of SIFT correspondences after 0.8 ratio test threshold: 
 [[20. 34. 37. ... 21. 35. 26.]
 [26. 28. 24. ... 23. 25. 22.]
 [22. 38. 45. ... 27. 38. 25.]
 ...
 [20. 27. 26. ... 14. 30. 21.]
 [27. 39. 32. ... 17. 34. 28.]
 [34. 33. 41. ... 30. 40. 15.]]
Predictions for 100 test Images:-
 ['30' '24' '22' '22' '79' '155' '149' '193' '21' '30' '140' '22' '34'
 '164' '30' '21' '30' '120' '30' '246' '87' '34' '140' '43' '140' '21'
 '89' '246' '239' '30' '246' '246' '34' '140' '23' '140' '120' '140' '209'
 '140' '272' '30' '24' '30' '51' '8' '21' '25' '140' '267' '34' '122' '30'
 '140' '22' '140' '246' '246' '120' '34' '50' '140' '140' '79' '74' '246'
 '24' '243' '10' '140' '246' '246' '246' '30' '246' '20' '34' '14' '125'
 '30' '246' '188' '246' '172' '120' '140' '34' '71' '34' '246' '158' '82'
 '33' '55' '94' '196' '190' '91' '31' '34']
Accuracy on NyalaData data: 5.00%



/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [17]:
# Nearest neigbour classifier using the similarity
classifier = KnnClassifier(k=1, database_labels=dataset_database_NyalaData.labels_string)
preds = classifier(sim)
print("Prediction \t", preds)
print("Ground truth \t", dataset_query_NyalaData.labels_string)

acc = sum(preds == dataset_query_NyalaData.labels_string) / len(dataset_query_NyalaData.labels_string)
print('\n Accuracy: ', acc*100,"%")

Prediction 	 ['30' '24' '22' '22' '79' '155' '149' '193' '21' '30' '140' '22' '34'
 '164' '30' '21' '30' '120' '30' '246' '87' '34' '140' '43' '140' '21'
 '89' '246' '239' '30' '246' '246' '34' '140' '23' '140' '120' '140' '209'
 '140' '272' '30' '24' '30' '51' '8' '21' '25' '140' '267' '34' '122' '30'
 '140' '22' '140' '246' '246' '120' '34' '50' '140' '140' '79' '74' '246'
 '24' '243' '10' '140' '246' '246' '246' '30' '246' '20' '34' '14' '125'
 '30' '246' '188' '246' '172' '120' '140' '34' '71' '34' '246' '158' '82'
 '33' '55' '94' '196' '190' '91' '31' '34']
Ground truth 	 ['19' '10' '57' '149' '58' '102' '147' '202' '114' '100' '149' '111' '33'
 '21' '65' '32' '36' '30' '131' '68' '85' '68' '19' '45' '10' '88' '111'
 '67' '242' '46' '145' '143' '30' '166' '23' '149' '37' '32' '209' '19'
 '23' '166' '30' '46' '51' '89' '114' '69' '242' '274' '181' '147' '51'
 '50' '223' '181' '47' '10' '7' '150' '46' '153' '40' '85' '94' '252'
 '230' '239' '11' '169' '113' '140' '60' '240' '102' '1

/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


**StripeSpotter Dataset**

In [18]:
# Download dataset (if not already downloaded)
datasets.StripeSpotter.get_data('../data/StripeSpotter')

# Load dataset metadata
metadata_StripeSpotter = datasets.StripeSpotter('../data/StripeSpotter')

# Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
transform = T.Compose([
    T.Resize([224, 224]),  # Resize the image
    # T.ToPILImage(),  # Convert tensor to PIL Image
    T.Grayscale(),  # Convert to grayscale
    # T.ToTensor(),  # Convert PIL Image to numpy array
    # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
])

# Create datasets with transformations
dataset = WildlifeDataset(metadata_StripeSpotter.df, metadata_StripeSpotter.root, transform=transform)
dataset_database_StripeSpotter = WildlifeDataset(metadata_StripeSpotter.df.iloc[100:], metadata_StripeSpotter.root, transform=transform)
dataset_query_StripeSpotter = WildlifeDataset(metadata_StripeSpotter.df.iloc[:100], metadata_StripeSpotter.root, transform=transform)

# Initialize SIFT extractor
sift = cv2.SIFT_create()
extractor_StripeSpotter = SIFTFeatures()

query_StripeSpotter, database_StripeSpotter = extractor_StripeSpotter(dataset_query_StripeSpotter), extractor_StripeSpotter(dataset_database_StripeSpotter)

print(f'First 5 query features shape: {[i.shape for i in query_StripeSpotter[:5]]}')
print(f'First 5 database features shape: {[i.shape for i in database_StripeSpotter[:5]]}')

DATASET StripeSpotter: DOWNLOADING STARTED.


data-20110718.zip: 71.7MB [00:01, 49.9MB/s]                              
data-20110718.z02: 100%|██████████| 78.6M/78.6M [00:01<00:00, 60.3MB/s]
data-20110718.z01: 100%|██████████| 78.6M/78.6M [00:01<00:00, 48.9MB/s]  


DATASET StripeSpotter: EXTRACTING STARTED.
 copying: data/
 copying: data/.DS_Store
 copying: data/images/
 copying: data/images/img-0000001.jpg
 copying: data/images/img-0000002.jpg
 copying: data/images/img-0000003.jpg
 copying: data/images/img-0000004.jpg
 copying: data/images/img-0000005.jpg
 copying: data/images/img-0000006.jpg
 copying: data/images/img-0000007.jpg
 copying: data/images/img-0000008.jpg
 copying: data/images/img-0000009.jpg
 copying: data/images/img-0000010.jpg
 copying: data/images/img-0000011.jpg
 copying: data/images/img-0000012.jpg
 copying: data/images/img-0000013.jpg
 copying: data/images/img-0000014.jpg
 copying: data/images/img-0000015.jpg
 copying: data/images/img-0000016.jpg
 copying: data/images/img-0000017.jpg
 copying: data/images/img-0000018.jpg
 copying: data/images/img-0000019.jpg
 copying: data/images/img-0000020.jpg
 copying: data/images/img-0000021.jpg
 copying: data/images/img-0000022.jpg
 copying: data/images/img-0000023.jpg
 copying: data/imag

100%|█████████████████████████████████████████████████████████████| 724/724 [00:31<00:00, 23.30it/s]

First 5 query features shape: [(414, 128), (610, 128), (704, 128), (295, 128), (396, 128)]
First 5 database features shape: [(410, 128), (556, 128), (548, 128), (216, 128), (571, 128)]


In [19]:
similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.7])
sim = similarity(query_StripeSpotter, database_StripeSpotter)[0.7]

print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)


classifier_StripeSpotter = KnnClassifier(k=1, database_labels=dataset_database_StripeSpotter.labels_string)
predictions_StripeSpotter = classifier_StripeSpotter(sim)

print("Predictions for 100 test Images:-\n",predictions_StripeSpotter)
accuracy_StripeSpotter = np.mean(dataset_query_StripeSpotter.labels_string == predictions_StripeSpotter)
print("Accuracy on StripeSpotter data: {:.2f}%".format(accuracy_StripeSpotter * 100))

100%|███████████████████████████████████████████████████████| 72400/72400 [00:42<00:00, 1692.70it/s]

Number of SIFT correspondences after 0.8 ratio test threshold: 
 [[45. 22. 23. ... 11. 31. 39.]
 [29. 26. 26. ... 46. 23. 25.]
 [52. 38. 36. ...  8. 30. 30.]
 ...
 [53. 32. 34. ... 13. 46. 34.]
 [38. 25. 45. ... 12. 48. 40.]
 [45. 38. 43. ... 14. 47. 45.]]
Predictions for 100 test Images:-
 ['09_212' 'S09_161' '04_008' '08_100' '09_200' 'OP05_625' '09_212'
 '02_149' '01_579' '08_034' '01_679' '08_078' '01_460' '10_002' '01_136'
 '09_212' '09_200' 'S09_111' '02_149' 'S09_069' '08_100' '01_363' '01_363'
 '08_078' '08_034' '01_805' '09_355' '08_085' 'OP05_546' '09_212' '08_100'
 'S09_088' '09_182' 'S09_215' '02_162' '01_230' '08_085' '01_661'
 '02_1026' '09_185' '01_136' '01_661' '01_661' '02_161' '01_165' '02_161'
 'S09_174' '02_162' '08_100' 'OP04_225' '02_149' 'S09_067' '09_200'
 '04_008' '01_661' '09_182' '01_805' '02_161' '10_002' '01_036' '02_149'
 '01_165' '09_212' 'S09_211' 'S09_215' '03_045' '01_579' 'OP04_225'
 '08_100' '08_100' '04_008' '01_661' 'OP05_625' 'OP05_625' '02_149'
 


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [20]:
# Nearest neigbour classifier using the similarity
classifier = KnnClassifier(k=1, database_labels=dataset_database_StripeSpotter.labels_string)
preds = classifier(sim)
print("Prediction \t", preds)
print("Ground truth \t", dataset_query_StripeSpotter.labels_string)

acc = sum(preds == dataset_query_StripeSpotter.labels_string) / len(dataset_query_StripeSpotter.labels_string)
print('\n Accuracy: ', acc*100,"%")

Prediction 	 ['09_212' 'S09_161' '04_008' '08_100' '09_200' 'OP05_625' '09_212'
 '02_149' '01_579' '08_034' '01_679' '08_078' '01_460' '10_002' '01_136'
 '09_212' '09_200' 'S09_111' '02_149' 'S09_069' '08_100' '01_363' '01_363'
 '08_078' '08_034' '01_805' '09_355' '08_085' 'OP05_546' '09_212' '08_100'
 'S09_088' '09_182' 'S09_215' '02_162' '01_230' '08_085' '01_661'
 '02_1026' '09_185' '01_136' '01_661' '01_661' '02_161' '01_165' '02_161'
 'S09_174' '02_162' '08_100' 'OP04_225' '02_149' 'S09_067' '09_200'
 '04_008' '01_661' '09_182' '01_805' '02_161' '10_002' '01_036' '02_149'
 '01_165' '09_212' 'S09_211' 'S09_215' '03_045' '01_579' 'OP04_225'
 '08_100' '08_100' '04_008' '01_661' 'OP05_625' 'OP05_625' '02_149'
 '01_230' '09_200' 'OP05_625' '01_661' 'OP04_225' '01_661' '08_078'
 'S09_174' '01_679' '01_661' 'S09_067' 'S09_111' 'S09_112' '09_212'
 '01_230' '02_161' '09_185' '04_008' '08_100' '01_460' '01_679' 'S09_111'
 '01_679' '01_036' 'S09_067']
Ground truth 	 ['09_212' 'S09_161' '08_0

/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


# **ON NEW UNSEEN DATASETS**

**DogFaceNet**

In [21]:
# Download dataset (if not already downloaded)
datasets.DogFaceNet.get_data('../data/DogFaceNet')

# Load dataset metadata
metadata_DogFaceNet = datasets.DogFaceNet('../data/DogFaceNet')


# Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
transform = T.Compose([
    T.Resize([224, 224]),  # Resize the image
    # T.ToPILImage(),  # Convert tensor to PIL Image
    T.Grayscale(),  # Convert to grayscale
    # T.ToTensor(),  # Convert PIL Image to numpy array
    # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
])

# Create datasets with transformations
dataset = WildlifeDataset(metadata_DogFaceNet.df, metadata_DogFaceNet.root, transform=transform)
dataset_database_DogFaceNet = WildlifeDataset(metadata_DogFaceNet.df.iloc[100:], metadata_DogFaceNet.root, transform=transform)
dataset_query_DogFaceNet = WildlifeDataset(metadata_DogFaceNet.df.iloc[:100], metadata_DogFaceNet.root, transform=transform)

# Initialize SIFT extractor
sift = cv2.SIFT_create()
extractor_DogFaceNet = SIFTFeatures()

query_DogFaceNet, database_DogFaceNet = extractor_DogFaceNet(dataset_query_DogFaceNet), extractor_DogFaceNet(dataset_database_DogFaceNet)

print(f'First 5 query features shape: {[i.shape for i in query_DogFaceNet[:5]]}')
print(f'First 5 database features shape: {[i.shape for i in database_DogFaceNet[:5]]}')


DATASET DogFaceNet: DOWNLOADING STARTED.


DogFaceNet_Dataset_224_1.zip: 75.6MB [00:03, 24.8MB/s]                              


DATASET DogFaceNet: EXTRACTING STARTED.
DATASET DogFaceNet: FINISHED.



100%|███████████████████████████████████████████████████████████| 8263/8263 [02:11<00:00, 62.70it/s]

First 5 query features shape: [(174, 128), (250, 128), (157, 128), (339, 128), (513, 128)]
First 5 database features shape: [(267, 128), (169, 128), (341, 128), (432, 128), (177, 128)]


In [23]:
similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.7])
sim = similarity(query_DogFaceNet, database_DogFaceNet)[0.7]

print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)


classifier_DogFaceNet = KnnClassifier(k=1, database_labels=dataset_database_DogFaceNet.labels_string)
predictions_DogFaceNet = classifier_DogFaceNet(sim)

print("Predictions for 100 test Images:-\n",predictions_DogFaceNet)
accuracy_DogFaceNet = np.mean(dataset_query_DogFaceNet.labels_string == predictions_DogFaceNet)
print("Accuracy on DogFaceNet data: {:.2f}%".format(accuracy_DogFaceNet * 100))


100%|█████████████████████████████████████████████████████| 826300/826300 [05:56<00:00, 2316.36it/s]

Number of SIFT correspondences after 0.8 ratio test threshold: 
 [[19. 14. 13. ... 18. 11. 11.]
 [15. 10. 23. ... 20. 12. 24.]
 [15. 19. 19. ... 13.  8. 17.]
 ...
 [14.  9. 24. ... 18.  7. 18.]
 [ 7. 14. 22. ...  9. 13. 20.]
 [16.  8. 18. ... 18.  9. 19.]]
Predictions for 100 test Images:-
 ['300' '1319' '1296' '223' '428' '1296' '943' '385' '974' '223' '1125'
 '300' '1296' '223' '300' '385' '1296' '385' '1125' '174' '1296' '1296'
 '151' '385' '1125' '1296' '1125' '300' '223' '385' '385' '385' '1296'
 '1296' '274' '96' '1296' '1296' '223' '1296' '751' '300' '300' '127'
 '961' '1296' '1296' '174' '1296' '424' '274' '943' '1296' '300' '1296'
 '300' '1125' '223' '1296' '11' '223' '1296' '96' '814' '1308' '631'
 '1296' '646' '1238' '223' '1296' '1296' '1125' '300' '1169' '458' '223'
 '149' '1296' '622' '167' '1125' '1296' '223' '1296' '1357' '1296' '223'
 '1296' '385' '1296' '943' '1296' '1296' '1125' '1296' '1296' '943' '1020'
 '1296']
Accuracy on DogFaceNet data: 11.00%



/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [24]:
# Nearest neigbour classifier using the similarity
classifier = KnnClassifier(k=1, database_labels=dataset_database_DogFaceNet.labels_string)
preds = classifier(sim)
print("Prediction \t", preds)
print("Ground truth \t", dataset_query_DogFaceNet.labels_string)

acc = sum(preds == dataset_query_DogFaceNet.labels_string) / len(dataset_query_DogFaceNet.labels_string)
print('\n Accuracy: ', acc*100,"%")

Prediction 	 ['300' '1319' '1296' '223' '428' '1296' '943' '385' '974' '223' '1125'
 '300' '1296' '223' '300' '385' '1296' '385' '1125' '174' '1296' '1296'
 '151' '385' '1125' '1296' '1125' '300' '223' '385' '385' '385' '1296'
 '1296' '274' '96' '1296' '1296' '223' '1296' '751' '300' '300' '127'
 '961' '1296' '1296' '174' '1296' '424' '274' '943' '1296' '300' '1296'
 '300' '1125' '223' '1296' '11' '223' '1296' '96' '814' '1308' '631'
 '1296' '646' '1238' '223' '1296' '1296' '1125' '300' '1169' '458' '223'
 '149' '1296' '622' '167' '1125' '1296' '223' '1296' '1357' '1296' '223'
 '1296' '385' '1296' '943' '1296' '1296' '1125' '1296' '1296' '943' '1020'
 '1296']
Ground truth 	 ['144' '182' '919' '1122' '428' '378' '1063' '426' '470' '405' '764'
 '1069' '3' '1339' '432' '673' '1075' '274' '296' '234' '1039' '991' '485'
 '425' '280' '407' '324' '1248' '1232' '896' '892' '448' '240' '251' '448'
 '930' '223' '210' '345' '763' '622' '1324' '158' '127' '313' '1254'
 '1139' '208' '193' '424' '29

/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T
